In [ ]:
# Imports and libs
import os
import os.path
import pathlib

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

import io
import scipy.misc
import numpy as np
import pandas as pd

import tensorflow as tf
tf.debugging.set_log_device_placement(False)
tf.get_logger().setLevel('ERROR')

import tensorflow_hub as hub

import PIL.Image as Image
import PIL.ImageColor as ImageColor
import PIL.ImageDraw as ImageDraw
import PIL.ImageFont as ImageFont

import six

from concurrent.futures import ThreadPoolExecutor

import collections

import ipywidgets as widgets
from IPython.display import display
from wordcloud import WordCloud

import time

# Load object detection lib
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# GLOBAL VARIABLES
USE_LEGACY = False
PATH_TO_LABELS = './tensorflow_models/research/object_detection/data/mscoco_label_map.pbtxt'
CATEGORY_INDEX = None
ALL_MODELS = {
    'EfficientDet' : 'https://tfhub.dev/tensorflow/efficientdet/d7/1',
    'Faster R-CNN' : 'https://tfhub.dev/tensorflow/faster_rcnn/inception_resnet_v2_1024x1024/1',
    'Faster R-CNN (OpenImages)' : 'https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1',
    'CenterNet' : 'https://tfhub.dev/tensorflow/centernet/resnet101v1_fpn_512x512/1'
}

In [ ]:
# The Faster R-CNN (Open Images) model requires use of legacy code, so set the value accordingly
def set_legacy(use_legacy):
    global USE_LEGACY, PATH_TO_LABELS, CATEGORY_INDEX
    
    USE_LEGACY = use_legacy
    if USE_LEGACY:
        tf.config.run_functions_eagerly(True)
        PATH_TO_LABELS = './tensorflow_models/research/object_detection/data/oid_v4_label_map.pbtxt'
    else:
        tf.config.run_functions_eagerly(False)
        PATH_TO_LABELS = './tensorflow_models/research/object_detection/data/mscoco_label_map.pbtxt'
    CATEGORY_INDEX = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
# Load model
def load_model(model_handle):
    if USE_LEGACY:
        hub_model = hub.load(model_handle).signatures['default']
    else:
        hub_model = hub.load(model_handle)
    
    print('Model loaded!')
    
    return hub_model

In [ ]:
# Get all valid image paths
# use image_type = 'colorized_artistic' for brighter/more daring colors
# use image_type = 'grayscale' for the original grayscale images
def get_valid_image_paths(path):
    valid_paths = []
    for image_path in os.listdir(path):
        if (image_path.endswith('.jpg')):
            # Collect image as valid path
            valid_paths.append(image_path)
    return valid_paths

# Load given image into numpy array using matplotlib imload
def load_valid_image(path, filename):
    img = mpimg.imread(path + filename, format='jpg')
    # Reshape image array to fit specifications required by tensorflow model
    (rows, columns, channels) = img.shape
    image = {
        'image': img.reshape((1, rows, columns, channels)),
        'filename': filename
    }
    return image

# The images are objects of the shape:
# {
#     'image': Actual Image Object,
#     'filename': Name of the original image
# }
def load_images(image_type='colorized_stable'):
    path = 'inputs/' + image_type + '/'
    valid_paths = get_valid_image_paths(path)
    
    # How many images were loaded so far
    progress_load = widgets.IntProgress(min=0, max=len(valid_paths), description='Loaded: ') # instantiate the bar
    load_label = widgets.Label(value='0 / ' + str(len(valid_paths)))
    display(widgets.HBox([progress_load, load_label])) # display the bar
    
    images = []
    for filename in valid_paths:
        images.append(load_valid_image(path, filename))
        
        progress_load.value += 1
        load_label.value = str(progress_load.value) + ' / ' + str(len(valid_paths))
        
    return images

In [ ]:
# Run inference with selected model on given images
# The array of images that was returned contains objects of the shape:
# {
#     'image': Actual Image Object,
#     'filename': Name of the original image
#     'detection': Detected objects on the image
# }
def run_inference(images, model):
    # How many images were used for inference so far
    progress_inference = widgets.IntProgress(min=0, max=len(images), description='Inferred: ')
    inference_label = widgets.Label(value='0 / ' + str(len(images)))
    display(widgets.HBox([progress_inference, inference_label]))
    
    images_with_detection = []
    
    for image in images:
        # Run object detection and save results
        if USE_LEGACY:
            converted = tf.image.convert_image_dtype(image['image'], tf.float32)
        else:
            converted = image['image']
        detected = model(converted)
        
        images_with_detection.append({
            'image': image['image'].copy(),
            'filename': image['filename'],
            'detection': {key:value.numpy() for key,value in detected.items()}
        })

        progress_inference.value += 1
        inference_label.value = str(progress_inference.value) + ' / ' + str(len(images))
    
    return images_with_detection

In [ ]:
# This is a shortened version of the tensorflow library function to visualize detections on images
# It has been shortened/adapted to support the use of variable font-sized based on the images dimensions
STANDARD_COLORS = [
    'AliceBlue', 'Chartreuse', 'Aqua', 'Aquamarine', 'Azure', 'Beige', 'Bisque',
    'BlanchedAlmond', 'BlueViolet', 'BurlyWood', 'CadetBlue', 'AntiqueWhite',
    'Chocolate', 'Coral', 'CornflowerBlue', 'Cornsilk', 'Crimson', 'Cyan',
    'DarkCyan', 'DarkGoldenRod', 'DarkGrey', 'DarkKhaki', 'DarkOrange',
    'DarkOrchid', 'DarkSalmon', 'DarkSeaGreen', 'DarkTurquoise', 'DarkViolet',
    'DeepPink', 'DeepSkyBlue', 'DodgerBlue', 'FireBrick', 'FloralWhite',
    'ForestGreen', 'Fuchsia', 'Gainsboro', 'GhostWhite', 'Gold', 'GoldenRod',
    'Salmon', 'Tan', 'HoneyDew', 'HotPink', 'IndianRed', 'Ivory', 'Khaki',
    'Lavender', 'LavenderBlush', 'LawnGreen', 'LemonChiffon', 'LightBlue',
    'LightCoral', 'LightCyan', 'LightGoldenRodYellow', 'LightGray', 'LightGrey',
    'LightGreen', 'LightPink', 'LightSalmon', 'LightSeaGreen', 'LightSkyBlue',
    'LightSlateGray', 'LightSlateGrey', 'LightSteelBlue', 'LightYellow', 'Lime',
    'LimeGreen', 'Linen', 'Magenta', 'MediumAquaMarine', 'MediumOrchid',
    'MediumPurple', 'MediumSeaGreen', 'MediumSlateBlue', 'MediumSpringGreen',
    'MediumTurquoise', 'MediumVioletRed', 'MintCream', 'MistyRose', 'Moccasin',
    'NavajoWhite', 'OldLace', 'Olive', 'OliveDrab', 'Orange', 'OrangeRed',
    'Orchid', 'PaleGoldenRod', 'PaleGreen', 'PaleTurquoise', 'PaleVioletRed',
    'PapayaWhip', 'PeachPuff', 'Peru', 'Pink', 'Plum', 'PowderBlue', 'Purple',
    'Red', 'RosyBrown', 'RoyalBlue', 'SaddleBrown', 'Green', 'SandyBrown',
    'SeaGreen', 'SeaShell', 'Sienna', 'Silver', 'SkyBlue', 'SlateBlue',
    'SlateGray', 'SlateGrey', 'Snow', 'SpringGreen', 'SteelBlue', 'GreenYellow',
    'Teal', 'Thistle', 'Tomato', 'Turquoise', 'Violet', 'Wheat', 'White',
    'WhiteSmoke', 'Yellow', 'YellowGreen'
]

def visualize_boxes_and_labels_on_image_array(
    image,
    boxes,
    classes,
    scores,
    category_index,
    use_normalized_coordinates=False,
    max_boxes_to_draw=20,
    min_score_thresh=.5,
    agnostic_mode=False,
    line_thickness=4,
    font_size=24,
    mask_alpha=.4,
    groundtruth_box_visualization_color='black'):

  # Create a display string (and color) for every box location, group any boxes
  # that correspond to the same location.
  box_to_display_str_map = collections.defaultdict(list)
  box_to_color_map = collections.defaultdict(str)
  box_to_track_ids_map = {}
  if not max_boxes_to_draw:
    max_boxes_to_draw = boxes.shape[0]
  for i in range(boxes.shape[0]):
    if max_boxes_to_draw == len(box_to_color_map):
      break
    if scores is None or scores[i] > min_score_thresh:
      box = tuple(boxes[i].tolist())
      if scores is None:
        box_to_color_map[box] = groundtruth_box_visualization_color
      else:
        display_str = ''
        if not agnostic_mode:
            if classes[i] in six.viewkeys(category_index):
              class_name = category_index[classes[i]]['name']
            else:
              class_name = 'N/A'
            display_str = str(class_name) + ' '
        box_to_display_str_map[box].append(display_str)
        if agnostic_mode:
          box_to_color_map[box] = 'DarkOrange'
        else:
          box_to_color_map[box] = STANDARD_COLORS[
              classes[i] % len(STANDARD_COLORS)]

  # Draw all boxes onto image.
  for box, color in box_to_color_map.items():
    ymin, xmin, ymax, xmax = box
    draw_bounding_box_on_image_array(
        image,
        ymin,
        xmin,
        ymax,
        xmax,
        color=color,
        thickness=line_thickness,
        font_size=font_size,
        display_str_list=box_to_display_str_map[box],
        use_normalized_coordinates=use_normalized_coordinates)

  return image

def draw_bounding_box_on_image_array(
    image,
    ymin,
    xmin,
    ymax,
    xmax,
    color='red',
    thickness=4,
    font_size=24,
    display_str_list=(),
    use_normalized_coordinates=True):

  image_pil = Image.fromarray(np.uint8(image)).convert('RGB')
  draw_bounding_box_on_image(image_pil, ymin, xmin, ymax, xmax, color,
                             thickness, font_size, display_str_list,
                             use_normalized_coordinates)
  np.copyto(image, np.array(image_pil))

def draw_bounding_box_on_image(
    image,
    ymin,
    xmin,
    ymax,
    xmax,
    color='red',
    thickness=4,
    font_size=24,
    display_str_list=(),
    use_normalized_coordinates=True):

  draw = ImageDraw.Draw(image)
  im_width, im_height = image.size
  if use_normalized_coordinates:
    (left, right, top, bottom) = (xmin * im_width, xmax * im_width,
                                  ymin * im_height, ymax * im_height)
  else:
    (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
  if thickness > 0:
    draw.line([(left, top), (left, bottom), (right, bottom), (right, top),
               (left, top)],
              width=thickness,
              fill=color)
  try:
    font = ImageFont.truetype('arial.ttf', font_size)
  except IOError:
    font = ImageFont.load_default()

  # If the total height of the display strings added to the top of the bounding
  # box exceeds the top of the image, stack the strings below the bounding box
  # instead of above.
  display_str_heights = [font.getsize(ds)[1] for ds in display_str_list]
  # Each display_str has a top and bottom margin of 0.05x.
  total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

  if top > total_display_str_height:
    text_bottom = top
  else:
    text_bottom = bottom + total_display_str_height
  # Reverse list and print from bottom to top.
  for display_str in display_str_list[::-1]:
    text_width, text_height = font.getsize(display_str)
    margin = np.ceil(0.05 * text_height)
    draw.rectangle(
        [(left, text_bottom - text_height - 2 * margin), (left + text_width,
                                                          text_bottom)],
        fill=color)
    draw.text(
        (left + margin, text_bottom - text_height - margin),
        display_str,
        fill='black',
        font=font)
    text_bottom -= text_height - 2 * margin

In [ ]:
# Determine how many objects were inferred and which object types for every image
# The array that is returned will contain one object of the following shape:
# {
#     'class': count,
#     'otherclass': othercount,
#     ....
# }
# for each of the given images
def get_detections_for_images(images):
    all_detections = []
    
    print(USE_LEGACY)
    for image in images:
        detections = image['detection']
        boxes = detections['detection_boxes'] if USE_LEGACY else detections['detection_boxes'][0]
        max_boxes_to_draw = boxes.shape[0]
        scores = detections['detection_scores'] if USE_LEGACY else detections['detection_scores'][0]
        coordinates = {}
        for i in range(min(max_boxes_to_draw, boxes.shape[0])):
            if scores[i] > 0.5:
                class_id = int(detections['detection_class_labels'][i]) if USE_LEGACY else int(detections['detection_classes'][0][i])
                class_name = CATEGORY_INDEX[class_id]['name']

                if class_name in coordinates:
                    coordinates[class_name] += 1
                else:
                    coordinates[class_name] = 1
        all_detections.append(coordinates)
        
    return all_detections

In [ ]:
def font_size_for_width(width):
    return int((width * 2) / 100)

def line_width_for_width(width):
    return int(np.log((width * 2) / 1000) * 5) + 1

# Draw detections on copied image
def draw_detections_on_images(images_with_detections):
    # How many detections were visualized so far
    progress_save = widgets.IntProgress(min=0, max=len(images_with_detections), description='Visualized: ')
    save_label = widgets.Label(value='0 / ' + str(len(images_with_detections)))
    display(widgets.HBox([progress_save, save_label]))
    
    for image in images_with_detections:
        # Determine font size based on image width
        (width, height, channels) = image['image'][0].shape

        # Draw detections
        visualize_boxes_and_labels_on_image_array(
              image['image'][0],
              image['detection']['detection_boxes'] if USE_LEGACY else image['detection']['detection_boxes'][0],
              image['detection']['detection_class_labels'].astype(int) if USE_LEGACY else image['detection']['detection_classes'][0].astype(int),
              image['detection']['detection_scores'] if USE_LEGACY else image['detection']['detection_scores'][0],
              CATEGORY_INDEX,
              use_normalized_coordinates=True,
              max_boxes_to_draw=200,
              # Change this if confidence score should be higher or lower
              min_score_thresh=.50,
              line_thickness=line_width_for_width(width),
              font_size=font_size_for_width(width),
              agnostic_mode=False
        )

        progress_save.value += 1
        save_label.value = str(progress_save.value) + ' / ' + str(len(images_with_detections))

In [ ]:
# Add output sub-directory named after the model that was used (if it doesn't already exist)
def save_images(images_with_detections, model_name):
    # How many images were saved so far
    progress_save = widgets.IntProgress(min=0, max=len(images_with_detections), description='Saved: ')
    save_label = widgets.Label(value='0 / ' + str(len(images_with_detections)))
    display(widgets.HBox([progress_save, save_label]))
    
    output_dir = 'outputs/' + model_name + '/'
    pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

    # Save images
    for image in images_with_detections:
        plt.imsave(output_dir + image['filename'], image['image'][0], format='jpg')

        progress_save.value += 1
        save_label.value = str(progress_save.value) + ' / ' + str(len(images_with_detections))

In [ ]:
# Run this if you want statistics on all inferences run for each model
# @param image_count: number of images that were used 
def get_statistics_for_detections(detections, image_count):
    dataFrame = pd.DataFrame(detections, index=range(1, image_count + 1))

    pathlib.Path('outputs/statistics/').mkdir(parents=True, exist_ok=True)
    dataFrame.to_csv('outputs/statistics/detections.csv')

In [ ]:
# Run this for charts
# @param image_count: number of images that were used
def generate_detection_charts(all_detections, image_count):
    plt.figure(figsize=(20, 16))
    
    # Total detections per image
    plt.subplot(221)
    total_detections = {}
    for model, detections in all_detections.items():
        model_detections = [np.sum(list(x.values())) for x in detections]
        total_detections[model] = np.sum(model_detections)
        plt.plot(model_detections)
    plt.ylabel('Detections per Image')
    plt.xlabel('Image Index')
    plt.legend(list(total_detections.keys()), loc='upper left')

    # Average detections per image
    plt.subplot(222)
    plt.ylabel('Average Detections per Image')
    plt.bar([name for name in total_detections.keys()], [value / image_count for value in total_detections.values()])

    # Total detections
    # plt.subplot(223)
    # plt.ylabel('Total Detections')
    # plt.bar([name for name in total_detections.keys()], [value for value in total_detections.values()])

    plt.savefig('outputs/statistics/statistics.svg', format='svg')
    plt.savefig('outputs/statistics/statistics.jpg', format='jpg')
    plt.savefig('outputs/statistics/statistics.pgf', format='pgf')

In [ ]:
def generate_wordcloud(detections):
    all_classes = [key for keys in [list(c.keys()) for c in detections] for key in keys]
    all_classes = ' '.join(all_classes)

    return WordCloud(background_color='white', width=1920, height=1080).generate(all_classes)

# Run this for a word cloud with the detected classes
def generate_wordclouds_for_all_models(all_detections):
    for model, detections in all_detections.items():
        wordcloud = generate_wordcloud(detections)

        # Show WordCloud
        print(model + ':')
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis(False)
        plt.show()

        # Save WordCloud
        wordcloud.to_file('outputs/statistics/' + model + '_wordcloud.jpg')
        plt.savefig('outputs/statistics/' + model + '_wordcloud.svg', format='svg')
        plt.savefig('outputs/statistics/' + model + '_wordcloud.pgf', format='pgf')

In [ ]:
def generate_wordclouds_by_dataset(all_detections):
    detections_mscoco = [v2 for k, v1 in all_detections.items() if k != 'Faster R-CNN (OpenImages)' for v2 in v1]
    
    mscoco_wc = generate_wordcloud(detections_mscoco)
    
    # Show WordCloud
    print('MS COCO:')
    plt.imshow(mscoco_wc, interpolation='bilinear')
    plt.axis(False)
    plt.show()

    # Save WordCloud
    mscoco_wc.to_file('outputs/statistics/mscoco_wordcloud.jpg')
    plt.savefig('outputs/statistics/mscoco_wordcloud.svg', format='svg')
    plt.savefig('outputs/statistics/mscoco_wordcloud.pgf', format='pgf')
    
    openimages_wc = generate_wordcloud(all_detections['Faster R-CNN (OpenImages)'])
    
    # Show WordCloud
    print('OpenImages V4:')
    plt.imshow(openimages_wc, interpolation='bilinear')
    plt.axis(False)
    plt.show()

    # Save WordCloud
    openimages_wc.to_file('outputs/statistics/openimages_wordcloud.jpg')
    plt.savefig('outputs/statistics/openimages_wordcloud.svg', format='svg')
    plt.savefig('outputs/statistics/openimages_wordcloud.pgf', format='pgf')

In [ ]:
# Draw first image
# @widgets.interact(i=(0, len(IMAGES)))
# def show_image_at(i=0):
#     plt.figure(figsize=(18,26))
#     plt.imshow(IMAGES[i]['image'][0])
#     plt.show()
    
# Display images with detection
# @widgets.interact(i=(0, len(IMAGES_WITH_DETECTION)))
# def show_image_at(i=0):
#     plt.figure(figsize=(24,32))
#     plt.imshow(IMAGES_WITH_DETECTION[i]['image'][0])
#     plt.show()